In [ ]:
# Fine tuning gpt2_medium model and use own data like company profile
#
# See also medium.com blog
# "GPT-2 Fine-Tuning Guide: Building a Chatbot for Your Company Profile"
# https://medium.com/@datatec.studio

In [ ]:
# Install python packages
# !pip install -r requirements.txt

In [1]:
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
import numpy as np
import sys

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Define environment variable, path of data, model name and device
# os.environ["HF_HOME"] = "/content/huggingface"  # Replace with your desired directory
# print("Please replace it with your hf access token:")
# os.environ["HF_HOME_TOKEN"] = "Please_replace_it_with_your_hf_access_token"

result_dir = 'resources/'
data_file_name = 'Winfrey'
data_file_path = f'../data/prompt_response/{data_file_name}.txt'

model_name = "gpt2" # gpt2-medium
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

cuda


In [4]:
# Write a python file to google driver
# Sample of json datasets
# You can also directly upload this code to your google driver
# The code write here in this way is for better understanding of whole project
# %%writefile chat_data.py

from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path: str, tokenizer):
        with open(path, encoding="utf-8") as f:
            self.data = f.readlines()#json.load(open(path, "r"))

        self.X = []
        for pair in self.data:
            pair = eval(pair)          
            self.X.append(f"<start> {pair['prompt']} <response>: {pair['response']} <end>")
        
        total_samples = len(self.X)  # Calculate the total number of samples
        print("total_samples", total_samples)
        # define samples amount
#         self.X = self.X[:500]
        print("Check the preprocessing for self.X[0]:")
        print(self.X[0])

        self.X_encoded = tokenizer(self.X, return_tensors="pt", max_length=30, padding="max_length", truncation=True)
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx]


In [5]:
# Download model, save model and tokernize to harddisk
## prepare tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

print(f'Number of tokens in tokenizer before adding our specific tokens: {len(tokenizer.get_vocab())}')

tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<start>",
                              "eos_token": "<end>"})

tokenizer.add_tokens(["<response>:"])
with open(f'../data/prompt_response/groupchat.txt', encoding="utf-8") as f:
    data = f.readlines()#json.load(open(path, "r"))

for pair in data:
    pair = eval(pair)
    tokenizer.add_tokens(pair['prompt'].split() + pair['response'].split())
    
print(f'Number of tokens in tokenizer after adding our specific tokens: {len(tokenizer.get_vocab())}')

## prepare model
### Specify the desired embedding size (must be a multiple of 8)
desired_embedding_size = 50264  # Change this to the desired size
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
### Resize the embedding layer to the desired size
model.resize_token_embeddings(len(tokenizer), desired_embedding_size)
model = model.to(device)

model.config.pad_token_id = tokenizer.eos_token_id + 1


## save tokenizer and model to harddisk
# tokenizer.save_pretrained(result_dir)
# model.save_pretrained(result_dir)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Number of tokens in tokenizer before adding our specific tokens: 50257
Number of tokens in tokenizer after adding our specific tokens: 52874


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
# ## load model and tokenizer from harddisk
# ### Load the GPT-2 tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained(result_dir)

# ### Load the GPT-2 model from the local folder
# model = GPT2LMHeadModel.from_pretrained(result_dir)
# model.to(device)

In [7]:
# Define infer and train function
def infer(inp_raw):
    inp_appended = "<start> " + inp_raw + " <response>: "
    inp = tokenizer(inp_appended, return_tensors="pt")
    X = inp["input_ids"].to(device)  # Use .to(device) method to move the tensor to the specified device
    a = inp["attention_mask"].to(device)  # Use .to(device) method here as well

    output = model.generate(X, attention_mask=a, max_length=100, num_return_sequences=1)
    output = tokenizer.decode(output[0]).replace('<pad>', '')
    output = output[output.find('<response>: ')+len('<response>: '):].replace('<response>:', '')
    if '<end>' in output:
        output = output[:output.find('<end>')]
    output = ' '.join(output.split())
    
    prompt_response = {
        'prompt': inp_raw,
        'response': output
    }

    return prompt_response


def train(chatData, model, optim):
    
    batches = len(chatData)
    total_loss = 0

    for i, (X, a) in tqdm(enumerate(chatData), total=len(chatData), desc="Training"):
        X = X.to(device)
        a = a.to(device)
        optim.zero_grad()
        loss = model(input_ids=X, attention_mask=a, labels=X).loss
        total_loss += loss.item()
        loss.backward()
        optim.step()

    print(infer("Any progress on the PA?"))
    print('========================================================')
    
    return total_loss

In [8]:
# from chat_data import ChatData

#Load ChatData, train model and optimizer
chatData = ChatData(data_file_path, tokenizer)
chatData = DataLoader(chatData, batch_size=1) # batch_size=64

model.train()

optim = Adam(model.parameters(), lr=1e-5)

total_samples 570
Check the preprocessing for self.X[0]:
<start> You named the group CSE 151B PA2. <response>: What should our group name be 😎 Ester and I used transformers for PA1 <end>


In [9]:
epochs = 200  # You can adjust the number of epochs as needed
training_loss = []
for epoch in range(epochs):
    start = time.time()
    print(f"Epoch {epoch} started")
    total_loss = train(chatData, model, optim)
    training_loss += [total_loss]
    end = time.time()
    print(f"Epoch {epoch} finished in {np.round((end - start) / 60, 2)} minutes | total_loss = {total_loss}")

Epoch 0 started


Training: 100%|██████████| 570/570 [00:22<00:00, 24.91it/s]


{'prompt': 'Any progress on the PA?', 'response': 'the PA the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the'}
Epoch 0 finished in 0.4 minutes | total_loss = 5005.146936416626
Epoch 1 started


Training: 100%|██████████| 570/570 [00:22<00:00, 25.06it/s]


{'prompt': 'Any progress on the PA?', 'response': 'the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the'}
Epoch 1 finished in 0.39 minutes | total_loss = 2845.416191339493
Epoch 2 started


...

{'prompt': 'Any progress on the PA?', 'response': 'the I I I I I I I I I I I I I I'}
Epoch 4 finished in 0.38 minutes | total_loss = 1817.9150650501251
Epoch 5 started


Training: 100%|██████████| 570/570 [00:21<00:00, 26.04it/s]


{'prompt': 'Any progress on the PA?', 'response': 'I the is the the the of the is the I is the the the the the the the the the the the the the the'}
Epoch 5 finished in 0.38 minutes | total_loss = 1727.6314778923988
Epoch 6 started


Training:  39%|███▉      | 222/570 [00:08<00:13, 25.63it/s]


KeyboardInterrupt: 

In [42]:
# Winfrey: 0.33749 for 100 epochs, 0.25071 for 200 epochs
# Jonathan 0.22917 for 200 epochs
# Ester 0.25416 for 200 epochs
# Jeremy 0.23813 for 200 epochs
# Samuel 0.27827 for 200 epochs
training_loss[-1] / len(chatData)

0.25416309113843716

In [ ]:
# inp = ""
# while True:
#     inp = input("Enter your input (press Enter when done): " + " " * 20)
#     print(infer(inp))

In [12]:
responses = []
for _ in range(1000):
    response = infer('')['response']
    responses.append(response)
    
responses

['I I I can I I I I I I I I I I I I I I',
 'I I I I I I I I I I I I I',
 'I I I I I I I I I I I']

In [43]:
# Save the model's state dictionary after training is complete
torch.save(model.state_dict(), f"models/{data_file_name}.pt")

# Simulate a Chat

In [64]:
ester = GPT2LMHeadModel.from_pretrained(model_name).to(device)
ester.resize_token_embeddings(len(tokenizer), desired_embedding_size)
ester.load_state_dict(torch.load("models/Ester.pt"))
ester.config.pad_token_id = tokenizer.eos_token_id + 1

winfrey = GPT2LMHeadModel.from_pretrained(model_name).to(device)
winfrey.resize_token_embeddings(len(tokenizer), desired_embedding_size)
winfrey.load_state_dict(torch.load("models/Winfrey.pt"))
winfrey.config.pad_token_id = tokenizer.eos_token_id + 1

jeremy = GPT2LMHeadModel.from_pretrained(model_name).to(device)
jeremy.resize_token_embeddings(len(tokenizer), desired_embedding_size)
jeremy.load_state_dict(torch.load("models/Jeremy.pt"))
jeremy.config.pad_token_id = tokenizer.eos_token_id + 1

jonathan = GPT2LMHeadModel.from_pretrained(model_name).to(device)
jonathan.resize_token_embeddings(len(tokenizer), desired_embedding_size)
jonathan.load_state_dict(torch.load("models/Jonathan.pt"))
jonathan.config.pad_token_id = tokenizer.eos_token_id + 1

samuel = GPT2LMHeadModel.from_pretrained(model_name).to(device)
samuel.resize_token_embeddings(len(tokenizer), desired_embedding_size)
samuel.load_state_dict(torch.load("models/Samuel.pt"))
samuel.config.pad_token_id = tokenizer.eos_token_id + 1

In [78]:
def group_infer(inp_raw, model):
    inp_appended = "<start> " + inp_raw + " <response>: "
    inp = tokenizer(inp_appended, return_tensors="pt")
    X = inp["input_ids"].to(device)  # Use .to(device) method to move the tensor to the specified device
    a = inp["attention_mask"].to(device)  # Use .to(device) method here as well

    output = model.generate(X, attention_mask=a, max_length=100, num_return_sequences=1)
    output = tokenizer.decode(output[0])
#     print(output)
    output = output[output.find('<response>: ')+len('<response>: '):].replace('<pad>', '')#.replace('<response>:', '')#.replace('<pad>', '')#.replace('<end>', '')
    if '<response>' in output:
        output = output[:output.find('<response>')]
    if '<end>' in output:
        output = output[:output.find('<end>')]
    output = ' '.join(output.split())
    
    prompt_response = {
        'prompt': inp_raw,
        'response': output
    }

    return prompt_response

In [113]:
speaker = 'Ester'
output = group_infer("wanna present with me?", eval(speaker.lower())) # How's it going?
with_name = f'{speaker}: ' + output['prompt']
convo = [with_name]
print(with_name)

people = ['Winfrey', 'Ester', 'Jeremy', 'Samuel', 'Jonathan']
for i in range(15):
    response = output['response']
    remaining_people = [person for person in people if person != speaker]
    speaker = np.random.choice(remaining_people)
    output = group_infer(response, eval(speaker.lower())) # assuming models are named ester, winfrey, ...
    with_name = f'{speaker}: ' + output['response']
    convo += [with_name]
    print(with_name)

Ester: wanna present with me?
Jeremy: but not need to be under 200
Winfrey: Not too sure, I only realized that very much later, the 71% val loss is already so far from me if we want, the loss is already I could just put these plots on a bunch of different type one, I
Jonathan: Like 6-10epochs, root library!
Winfrey: HT HT HT HT 0.9
Jeremy: oh btw a good idea. i think the main thing we need is more general
Ester: yes
Jeremy: ok i just pushed its under jeremy-unet, the code is still vv scuffed but idk more than the datahub ones i think? but like the cuda than the ones i thought urs would work cuz but mb there’s than the
Ester: 
Jeremy: ok i just pushed its under jeremy-unet, the code is still vv scuffed but idk oop im still vv using sequence length of 50 and using sequence of 50 and using sequence of 50 and using sequence of 50 and using sequence of 50 and using sequence of
Jonathan: 
Winfrey: Ok I’ll look into it in the morning 🤓 thanksss!!
Jonathan: do my write up part but if it go too

In [ ]:
convo

# Extra

In [44]:
# Define environment variable, path of data, model name and device
# os.environ["HF_HOME"] = "/content/huggingface"  # Replace with your desired directory
# print("Please replace it with your hf access token:")
# os.environ["HF_HOME_TOKEN"] = "Please_replace_it_with_your_hf_access_token"

result_dir = 'resources/'
data_file_name = 'Jeremy'
data_file_path = f'../data/prompt_response/{data_file_name}.txt'

model_name = "gpt2" # gpt2-medium
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

cuda


In [45]:
# Write a python file to google driver
# Sample of json datasets
# You can also directly upload this code to your google driver
# The code write here in this way is for better understanding of whole project
# %%writefile chat_data.py

from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path: str, tokenizer):
        with open(path, encoding="utf-8") as f:
            self.data = f.readlines()#json.load(open(path, "r"))

        self.X = []
        for pair in self.data:
            pair = eval(pair)          
            self.X.append(f"<start> {pair['prompt']} <response>: {pair['response']} <end>")
        
        total_samples = len(self.X)  # Calculate the total number of samples
        print("total_samples", total_samples)
        # define samples amount
#         self.X = self.X[:500]
        print("Check the preprocessing for self.X[0]:")
        print(self.X[0])

        self.X_encoded = tokenizer(self.X, return_tensors="pt", max_length=30, padding="max_length", truncation=True)
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx]


In [46]:
# Download model, save model and tokernize to harddisk
## prepare tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

print(f'Number of tokens in tokenizer before adding our specific tokens: {len(tokenizer.get_vocab())}')

tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<start>",
                              "eos_token": "<end>"})

tokenizer.add_tokens(["<response>:"])
with open(f'../data/prompt_response/groupchat.txt', encoding="utf-8") as f:
    data = f.readlines()#json.load(open(path, "r"))

for pair in data:
    pair = eval(pair)
    tokenizer.add_tokens(pair['prompt'].split() + pair['response'].split())
    
print(f'Number of tokens in tokenizer after adding our specific tokens: {len(tokenizer.get_vocab())}')

## prepare model
### Specify the desired embedding size (must be a multiple of 8)
desired_embedding_size = 50264  # Change this to the desired size
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
### Resize the embedding layer to the desired size
model.resize_token_embeddings(len(tokenizer), desired_embedding_size)
model = model.to(device)

model.config.pad_token_id = tokenizer.eos_token_id + 1


## save tokenizer and model to harddisk
# tokenizer.save_pretrained(result_dir)
# model.save_pretrained(result_dir)

Number of tokens in tokenizer before adding our specific tokens: 50257
Number of tokens in tokenizer after adding our specific tokens: 52874


In [47]:
# ## load model and tokenizer from harddisk
# ### Load the GPT-2 tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained(result_dir)

# ### Load the GPT-2 model from the local folder
# model = GPT2LMHeadModel.from_pretrained(result_dir)
# model.to(device)

In [48]:
# Define infer and train function
def infer(inp_raw):
    inp_appended = "<start> " + inp_raw + " <response>: "
    inp = tokenizer(inp_appended, return_tensors="pt")
    X = inp["input_ids"].to(device)  # Use .to(device) method to move the tensor to the specified device
    a = inp["attention_mask"].to(device)  # Use .to(device) method here as well

    output = model.generate(X, attention_mask=a, max_length=100, num_return_sequences=1)
    output = tokenizer.decode(output[0]).replace('<pad>', '')
    output = output[output.find('<response>: ')+len('<response>: '):].replace('<response>:', '')
    if '<end>' in output:
        output = output[:output.find('<end>')]
    output = ' '.join(output.split())
    
    prompt_response = {
        'prompt': inp_raw,
        'response': output
    }

    return prompt_response


def train(chatData, model, optim):
    
    batches = len(chatData)
    total_loss = 0

    for i, (X, a) in tqdm(enumerate(chatData), total=len(chatData), desc="Training"):
        X = X.to(device)
        a = a.to(device)
        optim.zero_grad()
        loss = model(input_ids=X, attention_mask=a, labels=X).loss
        total_loss += loss.item()
        loss.backward()
        optim.step()

    print(infer("Any progress on the PA?"))
    print('========================================================')
    
    return total_loss

In [49]:
# from chat_data import ChatData

#Load ChatData, train model and optimizer
chatData = ChatData(data_file_path, tokenizer)
chatData = DataLoader(chatData, batch_size=1) # batch_size=64

model.train()

optim = Adam(model.parameters(), lr=1e-5)

total_samples 164
Check the preprocessing for self.X[0]:
<start> What should our group name be 😎 Ester and I used transformers for PA1 <response>: lol anything works <end>


In [50]:
epochs = 200  # You can adjust the number of epochs as needed
training_loss = []
for epoch in range(epochs):
    start = time.time()
    print(f"Epoch {epoch} started")
    total_loss = train(chatData, model, optim)
    training_loss += [total_loss]
    end = time.time()
    print(f"Epoch {epoch} finished in {np.round((end - start) / 60, 2)} minutes | total_loss = {total_loss}")

Epoch 0 started


Training: 100%|██████████| 164/164 [00:06<00:00, 25.98it/s]


{'prompt': 'Any progress on the PA?', 'response': ''}
Epoch 0 finished in 0.12 minutes | total_loss = 2080.854018688202
Epoch 1 started


Training: 100%|██████████| 164/164 [00:06<00:00, 25.41it/s]


{'prompt': 'Any progress on the PA?', 'response': 'the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the'}
Epoch 1 finished in 0.12 minutes | total_loss = 1050.3898530006409
Epoch 2 started


...

{'prompt': 'Any progress on the PA?', 'response': 'nothing significant yet, gonna work on it rn oh btw take ur name'}
Epoch 197 finished in 0.12 minutes | total_loss = 38.559163719415665
Epoch 198 started


Training: 100%|██████████| 164/164 [00:06<00:00, 25.33it/s]


{'prompt': 'Any progress on the PA?', 'response': 'yea i will run it a single prediction and target i would pushed doing both 200 and 50 again and report back after rerunning'}
Epoch 198 finished in 0.12 minutes | total_loss = 39.798970848321915
Epoch 199 started


Training: 100%|██████████| 164/164 [00:06<00:00, 25.96it/s]


{'prompt': 'Any progress on the PA?', 'response': 'nothing significant yet, work on it later'}
Epoch 199 finished in 0.12 minutes | total_loss = 39.053896084427834


In [51]:
# Winfrey: 0.33749 for 100 epochs, 0.25071 for 200 epochs
# Jonathan 38.9592 for 200 epochs
training_loss[-1] / len(chatData)

0.2381335127099258

In [52]:
# Save the model's state dictionary after training is complete
torch.save(model.state_dict(), f"models/{data_file_name}.pt")

In [53]:
# Define environment variable, path of data, model name and device
# os.environ["HF_HOME"] = "/content/huggingface"  # Replace with your desired directory
# print("Please replace it with your hf access token:")
# os.environ["HF_HOME_TOKEN"] = "Please_replace_it_with_your_hf_access_token"

result_dir = 'resources/'
data_file_name = 'Samuel'
data_file_path = f'../data/prompt_response/{data_file_name}.txt'

model_name = "gpt2" # gpt2-medium
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

cuda


In [54]:
# Write a python file to google driver
# Sample of json datasets
# You can also directly upload this code to your google driver
# The code write here in this way is for better understanding of whole project
# %%writefile chat_data.py

from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path: str, tokenizer):
        with open(path, encoding="utf-8") as f:
            self.data = f.readlines()#json.load(open(path, "r"))

        self.X = []
        for pair in self.data:
            pair = eval(pair)          
            self.X.append(f"<start> {pair['prompt']} <response>: {pair['response']} <end>")
        
        total_samples = len(self.X)  # Calculate the total number of samples
        print("total_samples", total_samples)
        # define samples amount
#         self.X = self.X[:500]
        print("Check the preprocessing for self.X[0]:")
        print(self.X[0])

        self.X_encoded = tokenizer(self.X, return_tensors="pt", max_length=30, padding="max_length", truncation=True)
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx]


In [55]:
# Download model, save model and tokernize to harddisk
## prepare tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

print(f'Number of tokens in tokenizer before adding our specific tokens: {len(tokenizer.get_vocab())}')

tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<start>",
                              "eos_token": "<end>"})

tokenizer.add_tokens(["<response>:"])
with open(f'../data/prompt_response/groupchat.txt', encoding="utf-8") as f:
    data = f.readlines()#json.load(open(path, "r"))

for pair in data:
    pair = eval(pair)
    tokenizer.add_tokens(pair['prompt'].split() + pair['response'].split())
    
print(f'Number of tokens in tokenizer after adding our specific tokens: {len(tokenizer.get_vocab())}')

## prepare model
### Specify the desired embedding size (must be a multiple of 8)
desired_embedding_size = 50264  # Change this to the desired size
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
### Resize the embedding layer to the desired size
model.resize_token_embeddings(len(tokenizer), desired_embedding_size)
model = model.to(device)

model.config.pad_token_id = tokenizer.eos_token_id + 1


## save tokenizer and model to harddisk
# tokenizer.save_pretrained(result_dir)
# model.save_pretrained(result_dir)

Number of tokens in tokenizer before adding our specific tokens: 50257
Number of tokens in tokenizer after adding our specific tokens: 52874


In [56]:
# ## load model and tokenizer from harddisk
# ### Load the GPT-2 tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained(result_dir)

# ### Load the GPT-2 model from the local folder
# model = GPT2LMHeadModel.from_pretrained(result_dir)
# model.to(device)

In [57]:
# Define infer and train function
def infer(inp_raw):
    inp_appended = "<start> " + inp_raw + " <response>: "
    inp = tokenizer(inp_appended, return_tensors="pt")
    X = inp["input_ids"].to(device)  # Use .to(device) method to move the tensor to the specified device
    a = inp["attention_mask"].to(device)  # Use .to(device) method here as well

    output = model.generate(X, attention_mask=a, max_length=100, num_return_sequences=1)
    output = tokenizer.decode(output[0]).replace('<pad>', '')
    output = output[output.find('<response>: ')+len('<response>: '):].replace('<response>:', '')
    if '<end>' in output:
        output = output[:output.find('<end>')]
    output = ' '.join(output.split())
    
    prompt_response = {
        'prompt': inp_raw,
        'response': output
    }

    return prompt_response


def train(chatData, model, optim):
    
    batches = len(chatData)
    total_loss = 0

    for i, (X, a) in tqdm(enumerate(chatData), total=len(chatData), desc="Training"):
        X = X.to(device)
        a = a.to(device)
        optim.zero_grad()
        loss = model(input_ids=X, attention_mask=a, labels=X).loss
        total_loss += loss.item()
        loss.backward()
        optim.step()

    print(infer("Any progress on the PA?"))
    print('========================================================')
    
    return total_loss

In [58]:
# from chat_data import ChatData

#Load ChatData, train model and optimizer
chatData = ChatData(data_file_path, tokenizer)
chatData = DataLoader(chatData, batch_size=1) # batch_size=64

model.train()

optim = Adam(model.parameters(), lr=1e-5)

total_samples 65
Check the preprocessing for self.X[0]:
<start> I’ve done literally nothing but I’ll try for part 4 <response>: I also haven't done anything but I'll start today <end>


In [61]:
epochs = 200  # You can adjust the number of epochs as needed
training_loss = []
for epoch in range(epochs):
    start = time.time()
    print(f"Epoch {epoch} started")
    total_loss = train(chatData, model, optim)
    training_loss += [total_loss]
    end = time.time()
    print(f"Epoch {epoch} finished in {np.round((end - start) / 60, 2)} minutes | total_loss = {total_loss}")

Epoch 0 started


Training: 100%|██████████| 65/65 [00:02<00:00, 24.05it/s]


{'prompt': 'Any progress on the PA?', 'response': 'p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p p'}
Epoch 0 finished in 0.06 minutes | total_loss = 458.35883593559265
Epoch 1 started


Training: 100%|██████████| 65/65 [00:02<00:00, 24.71it/s]


{'prompt': 'Any progress on the PA?', 'response': 'the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the'}
Epoch 1 finished in 0.06 minutes | total_loss = 433.2690110206604
Epoch 2 started


...

{'prompt': 'Any progress on the PA?', 'response': 'no idea yet'}
Epoch 197 finished in 0.05 minutes | total_loss = 20.861480936408043
Epoch 198 started


Training: 100%|██████████| 65/65 [00:02<00:00, 27.22it/s]


{'prompt': 'Any progress on the PA?', 'response': 'no but one of the GANs sounds good'}
Epoch 198 finished in 0.05 minutes | total_loss = 19.126213908195496
Epoch 199 started


Training: 100%|██████████| 65/65 [00:02<00:00, 26.75it/s]


{'prompt': 'Any progress on the PA?', 'response': 'no idea how it knows tho'}
Epoch 199 finished in 0.05 minutes | total_loss = 18.08785840123892


In [62]:
# Winfrey: 0.33749 for 100 epochs, 0.25071 for 200 epochs
# Jonathan 38.9592 for 200 epochs
training_loss[-1] / len(chatData)

0.2782747446344449

In [63]:
# Save the model's state dictionary after training is complete
torch.save(model.state_dict(), f"models/{data_file_name}.pt")